# bison

Bison is a BNF lexer. But it turns out this is not enough.



## introduction


Tree-sitter also looks nice https://github.com/tree-sitter/tree-sitter-c

https://github.com/zaach/jison

https://en.m.wikipedia.org/wiki/GNU_Bison



In [ ]:
// placeholder for readme.md?


## jison parser



### TODO: select jison tree

Because parsing esprima code isn't enough! Let's use syntax tree features in any language. The Jison parse comes from Bison, which has lots of support

Works just like selectAst() except no esprima, generates the tree using jison instead, so it requires a language specifier either in a comment at the top of the code or in a parameter.  It can also read a file if it exists, so one more check in addition to reading a string, and the language can be derived from the file extension.



#### the code

select jison tree?

TODO: unfail this idea if `.y` files are found for the languages I want to use.



In [ ]:
var fs = require('fs');
var jison = require('jison');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])

function getParser(jisonFile) {
    var bnf
    if(fs.existsSync(jisonFile)) {
        bnf = fs.readFileSync(jisonFile, 'utf8')
    } else {
        bnf = importer.interpret(jisonFile).code
    }
    
    var parser = new jison.Parser(bnf)
    return parser
}


function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            code = fs.readFileSync(code)
            if(typeof language === 'undefined') {
                language = 
            }
        }
        var parser = getParser(language)
        try {
            code = esprima.parse(code, {range: true, tokens: true,
                                        comment: true, whitespace: true})
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // TODO: move this conversion to selectDom for all objects?
    //   No more selectTree because jsel and friends are inconsistent.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


### TODO: generate syntax tree by brute forcing tokens and lines

Source: https://stackoverflow.com/questions/8467150/how-to-get-abstract-syntax-tree-ast-out-of-jison-parser




In [1]:
function jisonAST(name, x) { this.name = name; this.x = x; }

// return the indented AST
jisonAST.prototype.get = function(indent){
  // create an indentation for level l
  function indentString(l) { var r=""; for(var i=0;i<l;i++){r+="  "}; return r }

  var r = indentString(indent) + "["+this.name+": ";
  var rem = this.x;
  if( rem.length == 1 && !(rem[0] instanceof jisonAST) ) r += "'"+rem[0]+"'"; 
  else for( i in rem ){ 
      if( rem[i] instanceof jisonAST ) r += "\n" + rem[i].get(indent+1);
      else { r += "\n" + indentString(indent+1); r += "'"+rem[i]+"'"; }
    }
  return r + "]";
}

function o( s ){
    r = "$$ = new yy.jisonAST('"+s+"',[";
    for( i = 1; i <= s.split(" ").length; i++ ){ r += "$"+i+"," }
    r = r.slice(0,-1) + "]);";
    return [s,r];
}

var Parser = require("jison").Parser;

// a grammar in JSON
var grammar = {
    "lex": {
        "rules": [
           ["\\s+", "/* skip whitespace */"],
           ["[a-f0-9]+", "return 'HEX';"]
        ]
    },
    "bnf": {
        // had to add a start/end, see below
        "start" : [ [ "hex_strings", "return $1" ] ],
        "hex_strings" :[ 
            o("hex_strings HEX"), 
            o("HEX") 
        ]
    }
};

var parser = new Parser(grammar);
// expose the AST object to Jison
parser.yy.jisonAST = jisonAST

console.log( parser.parse("adfe34bc e82a 43af").get(0) );

/*
expects
[hex_strings HEX: 
  [hex_strings HEX: 
    [HEX: 'adfe34bc']  
    'e82a']  
  '43af']
*/


Error: Cannot find module 'jison'
Require stack:
- /Users/briancullinan/jupyter_ops/Languages/[eval]